In [1]:
library(caret, quiet = TRUE)
library(base64enc)
library(httr, quiet = TRUE)

library(mlbench)


Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:

## multiclass classification in iris dataset:

data(BostonHousing)
BostonHousing$chas = as.numeric(BostonHousing$chas)

set.seed(1960)

dataset = BostonHousing[, -14] 

create_model  =  function() {

    model <- train(medv ~ ., data = BostonHousing, method = "rpart")    

    return(model)
}


In [3]:
model = create_model()
# cat(model$feature_names)
# print(model)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

In [4]:
pred_labels <- predict(model, BostonHousing[, -14] , type="raw")
df = data.frame(BostonHousing[,14])
names(df) = c("medv")
df$Estimator = pred_labels
df$Error = df$Estimator - df$medv
MAPE = mean(abs(df$Error / df$medv))
summary(df)
MAPE

      medv         Estimator         Error         
 Min.   : 5.00   Min.   :14.96   Min.   :-26.6502  
 1st Qu.:17.02   1st Qu.:14.96   1st Qu.: -2.6985  
 Median :21.20   Median :23.35   Median :  0.6529  
 Mean   :22.53   Mean   :22.53   Mean   :  0.0000  
 3rd Qu.:25.00   3rd Qu.:23.35   3rd Qu.:  3.1560  
 Max.   :50.00   Max.   :37.24   Max.   : 26.8382  

[1] 0.2063798

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)


WITH "DT_node_lookup" AS 
(SELECT "ADS"."KEY" AS "KEY", CASE WHEN ("ADS"."Feature_5" < 6.941) THEN CASE WHEN ("ADS"."Feature_12" >= 14.399999999999997) THEN 4 ELSE 5 END ELSE 3 END AS node_id_2 
FROM "INPUT_DATA" AS "ADS"), 
"DT_node_data" AS 
(SELECT "Values".nid AS nid, "Values"."E" AS "E" 
FROM (SELECT 3 AS nid, 37.23815789473685 AS "E" UNION ALL SELECT 4 AS nid, 14.956 AS "E" UNION ALL SELECT 5 AS nid, 23.349803921568643 AS "E") AS "Values"), 
"DT_Output" AS 
(SELECT "DT_node_lookup"."KEY" AS "KEY", "DT_node_lookup".node_id_2 AS node_id_2, "DT_node_data".nid AS nid, "DT_node_data"."E" AS "E" 
FROM "DT_node_lookup" LEFT OUTER JOIN "DT_node_data" ON "DT_node_lookup".node_id_2 = "DT_node_data".nid)
 SELECT "DT_Output"."KEY" AS "KEY", "DT_Output"."E" AS "Estimator" 
FROM "DT_Output"

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
df_sql = dataset
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

# df_sql

In [9]:
colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

[1] "Feature_0"  "Feature_1"  "Feature_2"  "Feature_3"  "Feature_4" 
 [6] "Feature_5"  "Feature_6"  "Feature_7"  "Feature_8"  "Feature_9" 
[11] "Feature_10" "Feature_11" "Feature_12" "KEY"

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out[order(df_sql_out$KEY),])

KEY,Estimator
1,23.34980
2,23.34980
3,37.23816
4,37.23816
5,37.23816
6,23.34980


In [11]:
# df_sql_out

# R RPART Output

In [12]:
estimator  =  predict(model, dataset, type = "raw")
df_r_out = data.frame(estimator)
names(df_r_out) = c("Estimator")

df_r_out$KEY = seq.int(nrow(dataset))
head(df_r_out)


Estimator,KEY
23.34980,1
23.34980,2
37.23816,3
37.23816,4
37.23816,5
23.34980,6


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Estimator_1,Estimator_2
1,23.34980,23.34980
2,23.34980,23.34980
3,37.23816,37.23816
4,37.23816,37.23816
5,37.23816,37.23816
6,23.34980,23.34980


In [14]:
df_merge$Error = df_merge$Estimator_1 - df_merge$Estimator_2
df_merge$AbsError = abs(df_merge$Error)
head(df_merge)


KEY,Estimator_1,Estimator_2,Error,AbsError
1,23.34980,23.34980,-7.105427e-15,7.105427e-15
2,23.34980,23.34980,-7.105427e-15,7.105427e-15
3,37.23816,37.23816,0.000000e+00,0.000000e+00
4,37.23816,37.23816,0.000000e+00,0.000000e+00
5,37.23816,37.23816,0.000000e+00,0.000000e+00
6,23.34980,23.34980,-7.105427e-15,7.105427e-15


In [15]:
df_merge_largest_errors = df_merge[df_merge$AbsError > 0.0001,]
df_merge_largest_errors

KEY,Estimator_1,Estimator_2,Error,AbsError


In [16]:
stopifnot(nrow(df_merge_largest_errors) == 0)

In [17]:
summary(df_sql_out)

      KEY          Estimator    
 Min.   :  1.0   Min.   :14.96  
 1st Qu.:127.2   1st Qu.:14.96  
 Median :253.5   Median :23.35  
 Mean   :253.5   Mean   :22.53  
 3rd Qu.:379.8   3rd Qu.:23.35  
 Max.   :506.0   Max.   :37.24  

In [18]:
summary(df_r_out)

   Estimator          KEY       
 Min.   :14.96   Min.   :  1.0  
 1st Qu.:14.96   1st Qu.:127.2  
 Median :23.35   Median :253.5  
 Mean   :22.53   Mean   :253.5  
 3rd Qu.:23.35   3rd Qu.:379.8  
 Max.   :37.24   Max.   :506.0  

In [19]:
summary(df_merge)

      KEY         Estimator_1     Estimator_2        Error           
 Min.   :  1.0   Min.   :14.96   Min.   :14.96   Min.   :-7.105e-15  
 1st Qu.:127.2   1st Qu.:14.96   1st Qu.:14.96   1st Qu.:-7.105e-15  
 Median :253.5   Median :23.35   Median :23.35   Median :-7.105e-15  
 Mean   :253.5   Mean   :22.53   Mean   :22.53   Mean   :-4.195e-15  
 3rd Qu.:379.8   3rd Qu.:23.35   3rd Qu.:23.35   3rd Qu.:-1.776e-15  
 Max.   :506.0   Max.   :37.24   Max.   :37.24   Max.   : 0.000e+00  
    AbsError        
 Min.   :0.000e+00  
 1st Qu.:1.776e-15  
 Median :7.105e-15  
 Mean   :4.195e-15  
 3rd Qu.:7.105e-15  
 Max.   :7.105e-15  